In [7]:
%matplotlib notebook
import matplotlib.pyplot as plt 
import numpy as np 
import scipy.sparse as sp 
from scipy import signal
from scipy.interpolate import interp1d
from mirnylib.numutils import observedOverExpectedWithMask, logbinsnew, zoomArray
import pandas as pd 
import cooler 
import pileups
import multiprocess as mp
import importlib as imp
import DNA_info
from functools import partial

In [97]:
res = 10000
arms = DNA_info.get_chromosome_arms('hg19', exclude=['Y','M'])
arms = [(x[0], (x[1]//res)*res, (x[2]//res)*res) for x in arms]
arms

[('chr1', 0, 123030000),
 ('chr1', 123030000, 249250000),
 ('chr2', 0, 93820000),
 ('chr2', 93820000, 243190000),
 ('chr3', 0, 92000000),
 ('chr3', 92000000, 198020000),
 ('chr4', 0, 51160000),
 ('chr4', 51160000, 191150000),
 ('chr5', 0, 47900000),
 ('chr5', 47900000, 180910000),
 ('chr6', 0, 60330000),
 ('chr6', 60330000, 171110000),
 ('chr7', 0, 59550000),
 ('chr7', 59550000, 159130000),
 ('chr8', 0, 45330000),
 ('chr8', 45330000, 146360000),
 ('chr9', 0, 48860000),
 ('chr9', 48860000, 141210000),
 ('chr10', 0, 40750000),
 ('chr10', 40750000, 135530000),
 ('chr11', 0, 53140000),
 ('chr11', 53140000, 135000000),
 ('chr12', 0, 36350000),
 ('chr12', 36350000, 133850000),
 ('chr13', 0, 17500000),
 ('chr13', 17500000, 115160000),
 ('chr14', 0, 17500000),
 ('chr14', 17500000, 107340000),
 ('chr15', 0, 18500000),
 ('chr15', 18500000, 102530000),
 ('chr16', 0, 36830000),
 ('chr16', 36830000, 90350000),
 ('chr17', 0, 23760000),
 ('chr17', 23760000, 81190000),
 ('chr18', 0, 16960000),
 ('chr1

In [2]:
cool = cooler.Cooler(f'/net/levsha/share/lab/U54_uC/hg19/coolers/library/U54-HFFc64DN-FA-DSG-MNase-R1-T2_hg19.{res}.cool')

NameError: name 'res' is not defined

In [ ]:
imp.reload(pileups)
mat_gen = pileups.fetchCooler(cool, arms)

In [ ]:
next(mat_gen)[1]

In [ ]:
loop_list = pd.read_csv('/net/levsha/share/sameer/U54/microc/pileups/loops/hiccups/GM12878_loop_list.csv')

In [90]:
len(loop_list)

9400

In [ ]:
cdf = pileups.chunkDataFrame(cool, arms, loop_list)

In [60]:
def loopPileup(chunked_feature_df, tup, columns=["ind1", "ind2"], pad=40):
    matrixAndBins, region = tup
    frame = chunked_feature_df[region]
    print(region)
    matrix, mask = matrixAndBins[0], matrixAndBins[1]
    m, n = matrix.shape
    matrix = pileups.sparseSymmetricOOE(matrix, mask)
    matrix = sp.csc_matrix(matrix, copy=False)
    locations1 = frame[columns[0]].values
    locations2 = frame[columns[1]].values

    total_PU = []
    mask_PU = []
    for loc1, loc2 in zip(locations1, locations2): 
        hi1 = loc1 + pad + 1
        lo1 = loc1 - pad
        hi2 = loc2 + pad + 1
        lo2 = loc2 - pad
        dm, dn = hi1 - lo1, hi2 - lo2
        
        out_of_bounds = False
        pad_left = pad_right = pad_bottom = pad_top = None
        if lo1 < 0:
            pad_bottom = -lo1
            out_of_bounds = True
        if lo2 < 0:
            pad_left = -lo2
            out_of_bounds = True
        if hi1 > m:
            pad_top = dm - (hi1 - m)
            out_of_bounds = True
        if hi2 > n:
            pad_right = dn - (hi2 - n)
            out_of_bounds = True
            
        if out_of_bounds:        
            i0 = max(lo1, 0)
            i1 = min(hi1, m)
            j0 = max(lo2, 0)
            j1 = min(hi2, n)
            
            submatrix = matrix[i0:i1, j0:j1].toarray().astype(float)
            submask1 = mask[i0:i1]
            submask2 = mask[j0:j1]
            outer = submask1[:,None] * submask2[None,:]
            submatrix[~outer] = 0 
            
            snippet =  np.full((dm, dn), 0.0)
            snip_mask = np.full((dm, dn), 0.0)
            
            snippet[pad_bottom:pad_top, 
                    pad_left:pad_right] = submatrix
            snip_mask[pad_bottom:pad_top, 
                    pad_left:pad_right] = outer
        else:
            submatrix = matrix[lo1:hi1, lo2:hi2].toarray().astype(float)
            submask1 = mask[lo1:hi1]
            submask2 = mask[lo2:hi2]
            outer = submask1[:,None] * submask2[None,:]
            submatrix[~outer] = 0
            
            snippet = submatrix
            snip_mask = outer

        nan_rows = np.sum(snippet, axis=0) == 0
        nan_cols = np.sum(snippet, axis=1) == 0
        snippet[nan_rows, :] = np.nan
        snippet[:, nan_cols] = np.nan
        total_PU.append(snippet)
        mask_PU.append(snip_mask)
    
    if not len(total_PU):
        total_PU = np.nan
        mask_PU = np.nan
    else:
        total_PU = np.dstack(tuple(total_PU))
        mask_PU = np.dstack(tuple(mask_PU))
    return total_PU, mask_PU

In [92]:
imp.reload(pileups)
res = 10000
loop_list = pd.read_csv('/net/levsha/share/sameer/U54/microc/pileups/loops/hiccups/GM12878_loop_list.csv')
arms = DNA_info.get_chromosome_arms('hg19', exclude=['Y','M'])
arms = [(x[0], (x[1]//res)*res, (x[2]//res)*res) for x in arms]
cool = cooler.Cooler(
    f'/net/levsha/share/lab/U54_uC/hg19/coolers/library/U54-HFFc64DN-FA-DSG-MNase-R1-T2_hg19.{res}.cool')
matrix_gen = pileups.fetchCooler(cool, arms)
cdf = pileups.chunkDataFrame(cool, arms, loop_list)
func = partial(pileups.loopPileup, cdf)
with mp.Pool(20) as p:
    result = list(p.map(func, [item for item in zip(matrix_gen, arms)]))

('chr1', 0, 123030000)
('chr1', 123030000, 249250000)
('chr2', 0, 93820000)
('chr2', 93820000, 243190000)
('chr3', 0, 92000000)
('chr3', 92000000, 198020000)
('chr4', 0, 51160000)
('chr4', 51160000, 191150000)
('chr5', 0, 47900000)
('chr5', 47900000, 180910000)
('chr6', 0, 60330000)
('chr6', 60330000, 171110000)
('chr7', 0, 59550000)
('chr7', 59550000, 159130000)
('chr8', 0, 45330000)
('chr8', 45330000, 146360000)
('chr9', 0, 48860000)
('chr9', 48860000, 141210000)
('chr10', 0, 40750000)
('chr10', 40750000, 135530000)
('chr11', 0, 53140000)
('chr11', 53140000, 135000000)
('chr12', 0, 36350000)
('chr13', 0, 17500000)
('chr12', 36350000, 133850000)
('chr13', 17500000, 115160000)
('chr14', 0, 17500000)
('chr14', 17500000, 107340000)
('chr15', 0, 18500000)
('chr15', 18500000, 102530000)
('chr16', 0, 36830000)
('chr16', 36830000, 90350000)
('chr17', 0, 23760000)
('chr17', 23760000, 81190000)
('chr18', 0, 16960000)
('chr18', 16960000, 78070000)
('chr19', 0, 26180000)
('chr19', 26180000, 5912

In [93]:
totals, masks = zip(*result)

In [94]:
masks = np.concatenate(tuple(total for total in masks if not np.all(np.isnan(total))), axis=2)
masks.shape

(81, 81, 9400)

In [127]:
def TADPileup(chunked_feature_df, pad, tup, columns=["ind1", "ind2"]):
    matrixAndBins, region = tup
    frame = chunked_feature_df[region]
    print(region)
    matrix, mask = matrixAndBins[0], matrixAndBins[1]
    m, n = matrix.shape
    matrix = pileups.sparseSymmetricOOE(matrix, mask)
    matrix = sp.csc_matrix(matrix, copy=False)
    starts = frame[columns[0]]
    ends = frame[columns[1]]
    centers = (starts+ends)//2

    total_PU = []
    mask_PU = []

    for mid in centers: 
        hi = mid + pad + 1
        lo = mid - pad
        
        dm = hi - lo
        out_of_bounds = False
        pad_left = pad_right = pad_bottom = pad_top = None
        if lo < 0:
            pad_bottom = -lo
            pad_left = -lo
            out_of_bounds = True
        if hi > m:
            pad_top = dm - (hi - m)
            out_of_bounds = True
        if hi > n:
            pad_right = dm - (hi - n)
            out_of_bounds = True
            
        if out_of_bounds:        
            i0 = max(lo, 0)
            i1 = min(hi, m)
            j0 = max(lo, 0)
            j1 = min(hi, n)
            
            submatrix = matrix[i0:i1, j0:j1].toarray().astype(float)
            submask1 = mask[i0:i1]
            submask2 = mask[j0:j1]
            outer = submask1[:,None] * submask2[None,:]
            submatrix[~outer] = 0 
            
            snippet =  np.full((dm, dm), 0.0)
            snip_mask = np.full((dm, dm), 0.0)
            
            snippet[pad_bottom:pad_top, 
                    pad_left:pad_right] = submatrix
            snip_mask[pad_bottom:pad_top, 
                    pad_left:pad_right] = outer
        else:
            submatrix = matrix[lo:hi, lo:hi].toarray().astype(float)
            submask = mask[lo:hi]
            outer = submask[:,None] * submask[None,:]
            submatrix[~outer] = 0
            
            snippet = submatrix
            snip_mask = outer

        nan_rows = np.sum(snippet, axis=0) == 0
        nan_cols = np.sum(snippet, axis=1) == 0
        snippet[nan_rows, :] = np.nan
        snippet[:, nan_cols] = np.nan
        total_PU.append(snippet)
        mask_PU.append(snip_mask)
    
    if not len(total_PU):
        total_PU = np.nan
        mask_PU = np.nan
    else:
        total_PU = np.dstack(tuple(total_PU))
        mask_PU = np.dstack(tuple(mask_PU))
    
    return total_PU, mask_PU

In [129]:
imp.reload(pileups)
res = 10000
tad_list = pd.read_csv('/net/levsha/share/sameer/U54/microc/pileups/tads/arrowhead/GM12878_tad_list.csv')
tad_list = tad_list.rename(columns={'chrom':'chrom1','start':'pos1', 'end':'pos2'})
tad_list['chrom2'] = tad_list['chrom1']
max_size = np.max((tad_list['pos2'] - tad_list['pos1']).values)
pad = np.ceil(max_size//res).astype(int)
arms = DNA_info.get_chromosome_arms('hg19', exclude=['Y','M'])
arms = [(x[0], (x[1]//res)*res, (x[2]//res)*res) for x in arms]
cool = cooler.Cooler(
    f'/net/levsha/share/lab/U54_uC/hg19/coolers/library/U54-HFFc64DN-FA-DSG-MNase-R1-T2_hg19.{res}.cool')
matrix_gen = pileups.fetchCooler(cool, arms)
cdf = pileups.chunkDataFrame(cool, arms, tad_list)
func = partial(TADPileup, cdf, pad)
with mp.Pool(20) as p:
    result = list(p.map(func, [item for item in zip(matrix_gen, arms)]))

('chr1', 0, 123030000)
('chr1', 123030000, 249250000)
('chr2', 0, 93820000)
('chr2', 93820000, 243190000)
('chr3', 0, 92000000)
('chr3', 92000000, 198020000)
('chr4', 0, 51160000)
('chr4', 51160000, 191150000)
('chr5', 0, 47900000)
('chr5', 47900000, 180910000)
('chr6', 0, 60330000)
('chr6', 60330000, 171110000)
('chr7', 0, 59550000)
('chr7', 59550000, 159130000)
('chr8', 0, 45330000)
('chr8', 45330000, 146360000)
('chr9', 0, 48860000)
('chr9', 48860000, 141210000)
('chr10', 0, 40750000)
('chr10', 40750000, 135530000)
('chr11', 0, 53140000)
('chr11', 53140000, 135000000)
('chr12', 0, 36350000)
('chr12', 36350000, 133850000)
('chr13', 0, 17500000)
('chr13', 17500000, 115160000)
('chr14', 0, 17500000)
('chr15', 0, 18500000)
('chr14', 17500000, 107340000)
('chr15', 18500000, 102530000)
('chr16', 0, 36830000)
('chr16', 36830000, 90350000)
('chr17', 0, 23760000)
('chr18', 0, 16960000)
('chr17', 23760000, 81190000)
('chr18', 16960000, 78070000)
('chr19', 0, 26180000)
('chr19', 26180000, 5912

MaybeEncodingError: Error sending result: '[(array([[[1.36228369, 2.35763594, 1.37063404, ..., 1.89256718,
         1.97797445, 1.63239968],
        [1.41390116, 0.62266157, 0.9317797 , ..., 0.91386473,
         1.0108359 , 1.00133739],
        [0.62350454, 0.35700689, 0.47174548, ..., 0.43216569,
         0.39164249, 0.32373924],
        ...,
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ]],

       [[1.41390116, 0.62266157, 0.9317797 , ..., 0.91386473,
         1.0108359 , 1.00133739],
        [2.07691128, 2.31094575, 2.90991988, ..., 1.96881323,
         2.01730826, 2.14873868],
        [0.41765031, 0.87006357, 1.31070929, ..., 1.16615092,
         1.06658627, 0.62690617],
        ...,
        [0.        , 0.        , 0.        , ..., 0.        ,
 

In [130]:
result

[(array([[[1.14770567, 0.88517455, 0.        , ..., 1.07887909,
           0.99090251, 4.36680836],
          [0.47581422, 1.65922145, 3.49390899, ..., 0.77344542,
           1.31735911, 3.61236729],
          [1.81650025, 1.06977946, 0.        , ..., 0.9121988 ,
           1.17910627, 1.03991296],
          ...,
          [0.        , 0.        , 0.        , ..., 0.        ,
           0.        , 0.        ],
          [0.        , 3.41393246, 0.        , ..., 0.        ,
           0.        , 2.88340042],
          [0.        , 0.        , 4.6617936 , ..., 0.        ,
           0.        , 2.86059425]],
  
         [[0.87791141, 0.68349619, 0.        , ..., 0.        ,
           2.61366685, 4.78233439],
          [1.45927436, 0.63897189, 0.        , ..., 0.79908682,
           2.087562  , 2.63874915],
          [0.93015948, 1.64977204, 0.        , ..., 0.        ,
           1.87075512, 2.28001479],
          ...,
          [2.83082007, 3.5386833 , 0.        , ..., 0.        ,
  

In [93]:
totals, masks = zip(*result)

In [94]:
masks = np.concatenate(tuple(total for total in masks if not np.all(np.isnan(total))), axis=2)
masks.shape

(81, 81, 9400)

In [ ]:
tad_list.shape

## Individual Regions

In [ ]:
a = fetchCooler(c,regions)
chunked_df = chunkDataFrame(c, regions, tads, columns=[("chr1","pos1"), ("chr2","pos2")])

# genome_mat = 0#np.zeros((2*pad+1,2*pad+1))
# genome_mask = 0#np.zeros((2*pad+1,2*pad+1))
# j=0

In [ ]:
matrixAndBins = next(a)

In [ ]:
frame = chunked_df[regions[0]]

matrix, bins = matrixAndBins[0], matrixAndBins[1]
M = matrix.shape[0]
mask = bins["mask"].values
matrix = sparseObservedOverExpected(matrix, mask)
matrix = sp.csc_matrix(matrix, copy=False)
starts = frame['pos1']
ends = frame['pos2']

In [ ]:
centers = (starts+ends)//2
sizes = ends-starts
pad = int(1.5*np.max(sizes))
total_PU = np.empty((len(locations1), 2*pad+1, 2*pad+1))
mask_PU = np.empty((len(locations1), 2*pad+1, 2*pad+1))

In [ ]:
# hist, bins = np.histogram(ends-starts, bins=50)
# width = 0.7 * (bins[1] - bins[0])
# center = (bins[:-1] + bins[1:])/2
# plt.bar(center, hist, align='center', width=width)
# plt.show()

In [ ]:


    for mid, wd in zip(centers, widths):
        
        if mid - pad < 0:
            continue
        if mid + pad + 1 > M:
            continue
        
        submatrix = matrix[mid-pad:mid+pad+1, mid-pad:mid+pad+1].todense()
        submask = mask[mid-pad:mid+pad+1]
        outer = submask[:,None] * submask[None,:]
        submatrix[~outer] = 0 
        assert submatrix[~outer].sum() == 0
        zoomedMatrix = zoomArray(submatrix, (81,81), sameSum=True, order=1)
        zoomedMask = zoomArray(outer, (81,81), sameSum=True, order=1)
        matrices = matrices + zoomedMatrix
        masks = masks + zoomedMask
    
    return matrices, masks

In [ ]:
imp.reload(pileups)
# pileup_list = []
# for i,region in zip(a, regions):
#     pileup_list.append(pileups.loopPileup((i, chunked_df[region], region)))
with Pool(20) as p:
        pileup_list = p.map(pileups.loopPileup, [(i, chunked_df[region], region) for i,region in zip(a, regions)])

In [ ]:
pileup_list

In [ ]:
totals, masks = list(zip(*pileup_list))

In [ ]:
totals = np.concatenate(tuple(totals), axis=0)
masks = np.concatenate(tuple(masks), axis=0)
totals.shape

In [ ]:
retrieved_loops = pd.concat(list(chunked_df.values()))
retrieved_loops

In [ ]:
loop_ind = loops.index.sort_values()
retrieved_ind = retrieved_loops.index.sort_values()

In [ ]:
print(len(retrieved_ind), len(loop_ind))

In [ ]:
miss_ind = np.setdiff1d(loop_ind, ind)
# miss_ind = np.append(miss_ind, 9401)

In [ ]:
while len(miss_ind) > 0:
    print(miss_ind)
    to_insert = miss_ind[miss_ind <= totals.shape[0]-1]
    miss_ind = miss_ind[miss_ind > totals.shape[0]-1]
    totals = np.insert(totals, to_insert, 0, axis=0)

In [ ]:
imp.reload(save_pileups)
res = save_pileups.genome_pileup(c, 'hg19', loops, pileups.loopPileup)

In [ ]:
totals.shape

In [ ]:
region = regions[j-1]
frame = chunkedLoops[region]
print(region)
matrices = 0#np.zeros((len(frame),2*pad+1,2*pad+1))
masks = 0#np.zeros((len(frame),2*pad+1,2*pad+1))
matrix, bins = i[0], i[1]
M = matrix.shape[0]
mask = bins["mask"].values

In [ ]:
mask

In [ ]:
matrix.shape

In [ ]:
#dense = matrix.todense()
#dense = observedOverExpectedWithMask(dense,mask[:,None]*mask[None,:])
mat = sparseObservedOverExpected(matrix, mask)

In [ ]:
matrix.todense()#[1000:8000,1000:8000]

In [ ]:
mat1 = mat.todense()[1000:8000,1000:8000]
fig = plt.figure(30)
ax = fig.add_subplot(111)
ax.set_title('Sparse OOE')
im = ax.imshow(mat1, vmax=4)
#im = ax.imshow(mask[1000:8000,None]*mask[None,1000:8000], vmax=4)
cax = fig.add_axes([0.98, 0.25, 0.005, 0.5])
fig.colorbar(im, cax=cax, orientation='vertical',ticklocation='left')
plt.show()

In [ ]:
mat = sp.csc_matrix(mat, copy=False)
starts = frame['ind1']
ends = frame['ind2']
centers = (starts+ends)//2
widths = ends-starts

for mid, wd in zip(centers, widths):
        pad = int(1.5*wd)
        if mid - pad < 0:
            continue
        if mid + pad + 1 > M:
            continue
        
        submatrix = mat[mid-pad:mid+pad+1, mid-pad:mid+pad+1].todense()
        submask = mask[mid-pad:mid+pad+1]
        outer = submask[:,None] * submask[None,:]
        submatrix[~outer] = 0 
        assert submatrix[~outer].sum() == 0
        zoomedMatrix = zoomArray(submatrix, (81,81), sameSum=True, order=1)
        zoomedMask = zoomArray(outer, (81,81), sameSum=True, order=1)
        matrices = matrices + zoomedMatrix
        masks = masks + zoomedMask
        
genome_mat = genome_mat + matrices
genome_mask = genome_mask + masks

In [ ]:
plt.figure(100)
plt.imshow(matrices/masks)
#plt.imshow(genome_mat/genome_mask)

In [ ]:
for k in range(21,42):
    plt.figure(k+1)
    plt.imshow(matrices[k,:,:])

In [ ]:
('chr3', 0, 92004854)

In [ ]:
import dekker

In [ ]:
dek = dekker.DekkerLib()
dek.data

In [ ]:
lps = dekker.get_loops(dek.data)

In [ ]:
cools = dekker.get_coolers(dek.data,'5000')

In [ ]:
dek.data.loc[46]

In [ ]:
plt.figure()
plt.imshow(lps.loc[46]['loops_pileup'])

In [ ]:
c = cools.loc[46]['5000']

In [ ]:
c = cooler.Cooler("/net/levsha/share/lab/dekkerHires/coolers/U54-160226-HFF-FA.5000.cool")
loops = pd.read_csv("/net/levsha/hic/DrosophilaSingleCell2015/GSE63525_GM12878_primary+replicate_HiCCUPS_looplist.txt", sep="\t")
loops["pos1"] = (loops["x1"] + loops["x2"]) // 2 
loops["pos2"] = (loops["y1"] + loops["y2"]) // 2 
loops = chr_cooler_format(c,loops)
tads = pd.read_csv("/net/levsha/hic/DrosophilaSingleCell2015/GSE63525_GM12878_primary+replicate_Arrowhead_domainlist.txt", sep="\t")
tads = chr_cooler_format(c,tads)
tads["pos1"] = tads["x1"]
tads["pos2"] = tads["x2"]

In [ ]:
from pileups import *

In [ ]:
regions = get_chrom_arms(c,'hg19')

a = fetchCooler(c,regions)
chunkedLoops = chunkDataFrame(c, regions, loops, columns=[("chr1","pos1"), ("chr2","pos2")])

with Pool(20) as p:
    pileup_list = p.map(loopPileup, [(i, chunkedLoops[region]) for i,region in zip(a, regions)])

In [ ]:
matrix = np.zeros((len(pileup_list), 81, 81))
mask = np.zeros((len(pileup_list), 81, 81))
for ind, lis in enumerate(pileup_list):
    p,m = lis
    matrix[ind,:,:] = p
    mask[ind,:,:] = m
    if not np.all(m==0):
        plt.figure(ind)
        plt.imshow(p/m)
    else:
        print(ind)
    

pileup = matrix/mask

In [ ]:
for i, sl in enumerate(pileup):
    if np.any(np.isnan(sl)):
        plt.figure(i)
        plt.imshow(sl)
        print( np.all(np.isnan(sl)))
    

In [ ]:
pile_total = np.nansum(pileup, axis=0)

In [ ]:
pile_total

In [ ]:
plt.figure()
plt.imshow(pile_total)

In [ ]:
import save_pileups

In [ ]:
filepaths = ['/net/levsha/share/lab/dekkerHires/coolers/U54-100716-HFF-EGS-DpnII.5000.cool','/net/levsha/share/lab/dekkerHires/coolers/U54-END-DSG-20160722-DpnII.5000.cool']
savepath = '/home/sameer/scalings_project/'

In [ ]:
%%time
imp.reload(save_pileups)
save_pileups.batch_process(filepaths, savepath)

In [ ]:
load = '/home/sameer/scalings_project/dekkerHires/coolers/loops/U54-100716-HFF-EGS-DpnII.5000.npy'
pu = np.load(load)

In [ ]:
plt.imshow(pu)

In [ ]:
import dekker

In [ ]:
dek = dekker.DekkerLib()
imp.reload(dekker)

In [ ]:
data = dek.filter_lib({})
data.loc[11]

In [ ]:
loops = dekker.get_loops(data)

In [ ]:
mat = loops['loops_pileup'][46]
plt.figure()
plt.imshow(mat)

In [ ]:
plt.figure()
plt.plot(np.diag(mat))

In [ ]:
plt.figure()
plt.plot(np.diag(np.fliplr(mat)))

In [ ]:
cools['5000'] = cools['5000'].apply(lambda x: x if 'weight' in x.bins()[0:5].columns else np.nan)

In [ ]:
cools= cools.dropna()
cools

In [ ]:
def chrom_corr(c):
    if 'chr' in c.chromnames[0]:
        regions = [('chr'+str(i),None,None) for i in range(1,23)]
        regions.append(('chrX', None, None))
        return regions
    else:
        regions = [(str(i),None,None) for i in range(1,23)]
        regions.append(('X', None, None))
        return regions

def save_pileup(mat, name):
    name = name[0:name.find('__')]+'5000'
    

In [ ]:
for c in cools['5000'].values:
    regions = chrom_corr(c)
    a = fetchCooler(c,regions)
    chunkedLoops = chunkDataFrame(c, regions, loops, columns=[("chr1","pos1"), ("chr2","pos2")])
    p1 = pileupOffDiagonal()
    
    for i,region in zip(a, regions):
        p1.processData(i, chunkedLoops[region])
        
    mat, mask =p1.getData()
    pileup = mat/mask
    
    
     

In [ ]:
imp.reload(save_pileups)
filepath = '/net/levsha/share/lab/dekkerHires/coolers/U54-100716-HFF-EGS-DpnII.10000.cool'
save_pileups.batch_process(filepath, '/home/sameer/scalings_project/')

In [ ]:
pileup = np.load('/home/sameer/scalings_project/dekkerHires/coolers/loops/U54-160226-HFF-FA.5000.npy')

In [ ]:
for i, sl in enumerate(pileup):
    if np.any(np.isnan(sl)):
        print(i, np.all(np.isnan(sl)))

In [ ]:
loops['chr1'].unique()